In [331]:
import pandas as pd

import glob
import functools
import urllib2

import pandas as pd
import json
import gspread
import pandas as pd
import os
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import numpy as np
import shutil

from IPython.core.display import HTML

In [332]:
!scp sauron.ucsd.edu:/rpool/www/for_encode/read_counts_2.txt .

read_counts_2.txt                   100%   27KB  27.3KB/s  27.3KB/s   00:00    


In [333]:
json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("Sheet1")
list_of_lists = ws.get_all_values()
manifest = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
manifest.is_encode = manifest.is_encode == "TRUE"
manifest.is_4000 = manifest.is_4000 == "TRUE"

manifest.method_Paper_flag = manifest.Method_Paper_flag == "TRUE" 
manifest['exp_id'] = manifest.ENCODE_ID.apply(lambda x: x.split("_")[0])

In [334]:
results = []
with open("read_counts_2.txt") as counts:
    while True:
        try:
            name_1 = counts.next().strip()
            read_count_1 = counts.next().strip()
            
            name_2 = counts.next().strip()
            read_count_2 = counts.next().strip()
            
            
            results.append({"name_1": name_1,
                            "name2": name_2,
                          "read_count_1": read_count_1,
                          "read_count_2": read_count_2,
})
        except:
            break
            
results = pd.DataFrame(results)
results['name'] = results.name2.apply(lambda x: x.split(".")[0])

In [335]:
results[~(results.read_count_1 == results.read_count_2)]

,name2,name_1,read_count_1,read_count_2,name


In [336]:
results = results.drop("read_count_2", axis=1)
results['rbp'] = results.name.apply(lambda x: x.split("_")[-1])

In [337]:
results["read_count_1"] = results.read_count_1.astype(int) / 4

In [338]:
manifest['fastq_name'] = manifest.apply(lambda x: "_".join([x.ENCODE_ID, x.RBP]), axis=1)

In [354]:
master_processing_table = pd.read_table("/home/elvannostrand/data/ENCODE/project_stuff/ENCODE_MASTER_ID_LIST_20160301_AllDatasets.txt", 
                                        sep="\t",
                                        skiprows=1,
                                        names=["UID", "RBP_gID", "CellLine", "RBP_ENSG", "Antibody", "Lot", "CLIP_ENCODEAccID", "CLIP_Rep1ENC", "CLIP_Rep2ENC", "CLIP_InputENC", "RNASEQ_ENCODEAccID",  "Duplicate_RNASEQ_ENCODEAccID", "RNASEQ_ControlENC", "RNASEQ_KDRep1Bam", "RNASEQ_KDRep2Bam", "RNASEQ_ControlRep1Bam", "RNASEQ_ControlRep2Bam", "bar"]
)

'ENCSR258QKO'

In [389]:
HTML(master_processing_table.sort_values("RBP_gID").to_html())

,UID,RBP_gID,CellLine,RBP_ENSG,Antibody,Lot,CLIP_ENCODEAccID,CLIP_Rep1ENC,CLIP_Rep2ENC,CLIP_InputENC,RNASEQ_ENCODEAccID,Duplicate_RNASEQ_ENCODEAccID,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,bar
407,624,AARS,K562,ENSG00000090861.11,A303-475A,001,ENCSR825SVO,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR599UDS,ENCSR599UDS,ENCSR572FFX,ENCFF402CEY.bam,ENCFF584BCB.bam,ENCFF719PLT.bam,ENCFF275ZEJ.bam,NaN
311,511,AATF,K562,ENSG00000108270.6,A301-032A,001,ENCSR639AUF,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR973QSV,ENCSR973QSV,ENCSR815CVQ,ENCFF509JMI.bam,ENCFF437YII.bam,ENCFF985IXD.bam,ENCFF461TSD.bam,NaN
296,496,AATF,HEPG2,ENSG00000108270.6,A301-032A,001,ENCSR498ISC,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR424YSV,ENCSR424YSV,ENCSR003EKR,ENCFF159JHT.bam,ENCFF800WEJ.bam,ENCFF383QRK.bam,ENCFF712XJM.bam,NaN
394,607,ABT1,K562,ENSG00000146109.3,A304-044A,001,ENCSR607IFN,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR251OLX,x,x,x,x,x,x,NaN
363,563,ABT1,K562,ENSG00000146109.3,A304-044A,001,ENCSR607IFN,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR251OLX,x,x,x,x,x,x,NaN
204,404,ACO1,K562,ENSG00000122729.14,RN039PW,001,ENCSR209YVB,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR423DIZ,x,x,x,x,x,x,NaN
196,396,ADAR,K562,ENSG00000160710.11,A303-883A,001,ENCSR620AMJ,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR164TLB,ENCSR164TLB,ENCSR667PLJ,ENCFF781RKY.bam,ENCFF148CSQ.bam,ENCFF804JHE.bam,ENCFF819ORB.bam,NaN
343,543,AGGF1,HEPG2,ENSG00000164252.8,A303-634A,001,ENCSR543TPH,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR606UYJ,x,x,x,x,x,x,NaN
337,537,AGGF1,K562,ENSG00000164252.8,A303-634A,001,ENCSR725ARB,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR812TLY,x,x,x,x,x,x,NaN
48,248,AGO1,K562,ENSG00000092847.6,RN028PW,002,ENCSR287DJT,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR268JDD,ENCSR268JDD,ENCSR084SCN,ENCFF244QXH.bam,ENCFF653PZL.bam,ENCFF178NNK.bam,ENCFF098BEA.bam,NaN


In [339]:
file_sizes = pd.merge(manifest, results, left_on="fastq_name", right_on="name") 

In [340]:
erics_numbers = pd.read_excel("/home/gpratt/Dropbox/NatMethodsPaper/SupplementaryTable2.xlsx", "Sheet1", skiprows=2)
erics_numbers['rbp_and_gene_id'] = erics_numbers.apply(lambda x: "_".join([x['RBP'], x['Cell line']]), axis=1)

In [427]:
file_sizes['foo'] = file_sizes.RBP + "_" + file_sizes.cell_type
bar  = file_sizes.drop([u'inline_1', u'inline_2',
       u'index_1', u'index_2', u'Lane', u'file_location', u'unmerged_location',
       u'original_file_name', u'is_encode', u'hiseq_run_date',
       u'randomer_length', u'', u'Method_Paper_flag', u'species', u'is_4000',
       u'exp_id', u'fastq_name', 'Hiseq_file_name'],axis=1)

bar = bar[~bar.ENCODE_ID.apply(lambda x: "INPUT" in x)]
bar = bar[~bar.ENCODE_ID.apply(lambda x: "4000" in x)]
bar = bar.drop(74, axis=0)
bar['UID'] = bar.ENCODE_ID.apply(lambda x:x.split("_")[0])

In [433]:
result = []
for name, df in bar.groupby(["UID"]):
    result.append({"encode_id": int(df.iloc[0]['ENCODE_ID'].split("_")[0]), "name": df.iloc[0]['foo'], "rep1": df.iloc[0].read_count_1, "rep2": df.iloc[1].read_count_1})

result = pd.DataFrame(result)
result = pd.merge(result, master_processing_table, left_on="encode_id", right_on="UID")

In [434]:
master_table = pd.merge(erics_numbers, result, left_on="eCLIP accession", right_on="CLIP_ENCODEAccID", how="outer")

In [435]:
HTML(master_table.to_html())

,RBP,Cell line,Antibody catalog #,Antibody lot #,eCLIP accession,SMInput accession,Replicate accession,Input read #,Uniquely mapped read #,Usable read #,% usable (out of uniquely mapped),Replicate accession.1,Input read #.1,Uniquely mapped read #.1,Usable read #.1,% usable (out of uniquely mapped).1,rbp_and_gene_id,encode_id,name,rep1,rep2,UID,RBP_gID,CellLine,RBP_ENSG,Antibody,Lot,CLIP_ENCODEAccID,CLIP_Rep1ENC,CLIP_Rep2ENC,CLIP_InputENC,RNASEQ_ENCODEAccID,Duplicate_RNASEQ_ENCODEAccID,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,bar
0,AUH,HepG2,RN037P,1,ENCSR334QFR,ENCSR821VMD,ENCLB381KLL,13553232,3558228,2664051,0.748702,ENCLB814JNF,15493437,3354237,2520409,0.751411,AUH_HepG2,280,AUH_HepG2,13553232,15493437,280,AUH,HEPG2,ENSG00000148090.7,RN037P,001,ENCSR334QFR,ENCLB381KLL,ENCLB814JNF,ENCLB828TBY,ENCSR409CSO,x,x,x,x,x,x,NaN
1,AUH,K562,RN037P,1,ENCSR541QHS,ENCSR943ZKM,ENCLB274RLM,13631589,1365661,740559,0.542271,ENCLB441HJU,13850133,1529778,500638,0.327262,AUH_K562,246,AUH_K562,13631589,13850133,246,AUH,K562,ENSG00000148090.7,RN037P,001,ENCSR541QHS,ENCLB274RLM,ENCLB441HJU,ENCLB320EKU,ENCSR395FYF,x,x,x,x,x,x,NaN
2,BCCIP,HepG2,A302-196A,1,ENCSR485QCG,ENCSR655BRZ,ENCLB473MOT,8944087,2838390,2470161,0.870268,ENCLB896QXF,8996636,3203943,2791263,0.871196,BCCIP_HepG2,230,BCCIP_HepG2,8944087,8996636,230,BCCIP,HEPG2,ENSG00000107949.12,A302-196A,001,ENCSR485QCG,ENCLB473MOT,ENCLB896QXF,ENCLB822BHO,ENCSR570CWH,ENCSR570CWH,ENCSR856ZRV,ENCFF596JEB.bam,ENCFF022PCS.bam,ENCFF038LUH.bam,ENCFF705LNP.bam,NaN
3,BUD13,HepG2,A303-320A,1,ENCSR830BSQ,ENCSR777PDA,ENCLB482NEH,32516195,8509346,5772548,0.678377,ENCLB719MGT,32045914,7769078,5610156,0.722113,BUD13_HepG2,497,BUD13_,32516195,32045914,497,BUD13,HEPG2,ENSG00000137656.7,A303-320A,001,ENCSR830BSQ,ENCLB482NEH,ENCLB719MGT,ENCLB664POM,ENCSR382QKD,x,x,x,x,x,x,NaN
4,BUD13,K562,A303-320A,1,ENCSR663WES,ENCSR736LKF,ENCLB679DRC,28367330,11323686,9667173,0.853713,ENCLB612WYL,28502666,11333288,8746771,0.771777,BUD13_K562,508,BUD13_,28367330,28502666,508,BUD13,K562,ENSG00000137656.7,A303-320A,001,ENCSR663WES,ENCLB679DRC,ENCLB612WYL,ENCLB279RPQ,ENCSR267RHP,x,x,x,x,x,x,NaN
5,CPSF6,K562,A301-356A,1,ENCSR532VUB,ENCSR249WNK,ENCLB639LHO,11832728,4614808,2815791,0.610164,ENCLB853UAV,11492161,5243635,2281621,0.435122,CPSF6_K562,258,CPSF6_K562,11832728,11492161,258,CPSF6,K562,ENSG00000111605.12,A301-356A,001,ENCSR532VUB,ENCLB639LHO,ENCLB853UAV,ENCLB998LZP,ENCSR895BTE,ENCSR895BTE,ENCSR913CAE,ENCFF440ILL.bam,ENCFF692ZZQ.bam,ENCFF309OOI.bam,ENCFF209PLP.bam,NaN
6,CSTF2T,HepG2,A301-487A,3,ENCSR919HSE,ENCSR391GLS,ENCLB109CUT,21035526,12258000,10516980,0.857969,ENCLB870VMU,16710536,10768032,10122009,0.940005,CSTF2T_HepG2,292,CSTF2T_HepG2,21035526,16710536,292,CSTF2T,HEPG2,ENSG00000177613.7,A301-487A,003,ENCSR919HSE,ENCLB109CUT,ENCLB870VMU,ENCLB523ZOX,ENCSR914WQV,ENCSR914WQV,ENCSR003EKR,ENCFF352MST.bam,ENCFF187JOD.bam,ENCFF383QRK.bam,ENCFF712XJM.bam,NaN
7,CSTF2T,K562,A301-487A,1,ENCSR840DRD,ENCSR722LZB,ENCLB675HEB,10164425,6527270,5794921,0.887802,ENCLB570DQA,15979572,9974471,9117670,0.914101,CSTF2T_K562,275,CSTF2T_K562,10164425,15979572,275,CSTF2T,K562,ENSG00000177613.7,A301-487A,001,ENCSR840DRD,ENCLB675HEB,ENCLB570DQA,ENCLB513ZXE,ENCSR286OKW,ENCSR286OKW,ENCSR815CVQ,ENCFF743GOG.bam,ENCFF110CVQ.bam,ENCFF985IXD.bam,ENCFF461TSD.bam,NaN
8,DDX3X,HepG2,A330-474A,1,ENCSR648LAH,ENCSR090VXV,ENCLB933GRD,27339086,7006445,3582239,0.511278,ENCLB436IZW,25693228,6612307,1822067,0.275557,DDX3X_HepG2,492,DDX3X_,27339086,25693228,492,DDX3X,HEPG2,ENSG00000215301.5,A330-474A,001,ENCSR648LAH,ENCLB933GRD,ENCLB436IZW,ENCLB086ERJ,ENCSR637JLM,ENCSR637JLM,ENCSR237YZT,ENCFF201FHG.bam,ENCFF977MZX.bam,ENCFF052HTH.bam,ENCFF857QSU.bam,NaN
9,DDX3X,K562,A300-474A,1,ENCSR930BZL,ENCSR028CRL,ENCLB780MPM,13967203,4239302,2874105,0.677967,ENCLB657WFG,12378451,3541674,2687447,0.758807,DDX3X_K562,350,DDX3X_,13967203,12378451,350,DDX3X,K562,ENSG00000215301.5,A300-474A,001,ENCSR930BZL,ENCLB780MPM,ENCLB657WFG,ENCLB5

In [436]:
HTML(master_table[~(master_table['Input read #'].astype(float) == master_table['rep1'].astype(float))].to_html())

,RBP,Cell line,Antibody catalog #,Antibody lot #,eCLIP accession,SMInput accession,Replicate accession,Input read #,Uniquely mapped read #,Usable read #,% usable (out of uniquely mapped),Replicate accession.1,Input read #.1,Uniquely mapped read #.1,Usable read #.1,% usable (out of uniquely mapped).1,rbp_and_gene_id,encode_id,name,rep1,rep2,UID,RBP_gID,CellLine,RBP_ENSG,Antibody,Lot,CLIP_ENCODEAccID,CLIP_Rep1ENC,CLIP_Rep2ENC,CLIP_InputENC,RNASEQ_ENCODEAccID,Duplicate_RNASEQ_ENCODEAccID,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,bar
29,HNRNPC,HepG2,RN052PW,1,ENCSR550DVK,ENCSR497ANA,ENCLB094PVE,31226412,15430668,15153574,0.982043,ENCLB392RHM,22156717,12927959,10173752,0.786957,HNRNPC_HepG2,203,HNRNPC_HepG2,5428248,22156717,203,HNRNPC,HEPG2,ENSG00000092199.13,RN052PW,001,ENCSR550DVK,ENCLB094PVE,ENCLB392RHM,ENCLB104YKZ,ENCSR052IYH,ENCSR052IYH,ENCSR305XWT,ENCFF031QZG.bam,ENCFF531AUK.bam,ENCFF460AAE.bam,ENCFF058QDT.bam,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,290,CPSF6_,16473039,24346353,290,CPSF6,HEPG2,ENSG00000111605.12,A301-356A,001,ENCSR693HXO,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR676EKU,x,x,x,x,x,x,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,376,SRSF7_,4296108,9323870,376,SRSF7,K562,ENSG00000115875.14,RN079PW,001,ENCSR468FSW,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR464ADT,ENCSR464ADT,ENCSR129RWD,ENCFF583OAC.bam,ENCFF485YBJ.bam,ENCFF970PYT.bam,ENCFF019XTR.bam,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414,PPIL4_,22165168,17136913,414,PPIL4,K562,ENSG00000131013.3,GTX117014,40555,ENCSR197INS,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR556FNN,x,x,x,x,x,x,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,425,AKAP8L_,31711856,18213279,425,AKAP8L,K562,ENSG00000011243.13,GTX115831,40296,ENCSR206RXT,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR809ISU,x,x,x,x,x,x,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437,SFPQ_,17821425,10668494,437,SFPQ,HEPG2,ENSG00000116560.6,A301-321A,001,ENCSR965DLL,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR782MXN,x,x,x,x,x,x,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460,FASTKD2_,12082491,16233147,460,FASTKD2,K562,ENSG00000118246.9,A303-788A,001,ENCSR887FHF,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR608IAI,ENCSR608IAI,ENCSR084SCN,ENCFF635XVZ.bam,ENCFF312ZOZ.bam,ENCFF178NNK.bam,ENCFF098BEA.bam,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,461,LARP4_,13871381,18367153,461,LARP4,K562,ENSG00000161813.16,A303-900A,001,ENCSR888YTT,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR866XLI,x,x,x,x,x,x,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,470,MTPAP_,42501715,24125815,470,MTPAP,K562,ENSG00000107951.8,A303-904A,001,ENCSR200DKE,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR631RFX,x,x,x,x,x,x,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483,DDX24_,20559528,14663482,483,DDX24,K562,ENSG00000089737.11,A300-696A,001,ENCSR999WKT,MISSING_ACCESSION,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR067LLB,ENCSR067LLB,ENCSR913CAE,ENCFF084KMK.bam,ENCFF626FFP.bam,ENCFF309OOI.bam,ENCFF209PLP.bam,NaN


In [437]:
master_table[~(master_table['Input read #.1'].astype(float) == master_table['rep2'].astype(float))]

,RBP,Cell line,Antibody catalog #,Antibody lot #,eCLIP accession,SMInput accession,Replicate accession,Input read #,Uniquely mapped read #,Usable read #,...,CLIP_Rep2ENC,CLIP_InputENC,RNASEQ_ENCODEAccID,Duplicate_RNASEQ_ENCODEAccID,RNASEQ_ControlENC,RNASEQ_KDRep1Bam,RNASEQ_KDRep2Bam,RNASEQ_ControlRep1Bam,RNASEQ_ControlRep2Bam,bar
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR676EKU,x,x,x,x,x,x,NaN
103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR464ADT,ENCSR464ADT,ENCSR129RWD,ENCFF583OAC.bam,ENCFF485YBJ.bam,ENCFF970PYT.bam,ENCFF019XTR.bam,NaN
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR556FNN,x,x,x,x,x,x,NaN
105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR809ISU,x,x,x,x,x,x,NaN
106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR782MXN,x,x,x,x,x,x,NaN
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR608IAI,ENCSR608IAI,ENCSR084SCN,ENCFF635XVZ.bam,ENCFF312ZOZ.bam,ENCFF178NNK.bam,ENCFF098BEA.bam,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR866XLI,x,x,x,x,x,x,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR631RFX,x,x,x,x,x,x,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR067LLB,ENCSR067LLB,ENCSR913CAE,ENCFF084KMK.bam,ENCFF626FFP.bam,ENCFF309OOI.bam,ENCFF209PLP.bam,NaN
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MISSING_ACCESSION,MISSING_ACCESSION,ENCSR081XRA,ENCSR081XRA,ENCSR344XID,ENCFF960VQB.bam,ENCFF375HRX.bam,ENCFF092XAP.bam,ENCFF265ZZB.bam,NaN


In [347]:
foo = pd.read_table("/home/gpratt/ad-hoc/X2A_files.txt", header=None)

In [251]:
",".join(set(foo[0].apply(lambda x: x.split(".")[0])))


'517_CLIP_S13_L002_R1_001,LNG2-N_CLIP_S76_L007_R2_001,LNG2-M_INPUT_S77_L007_R1_001,577_CLIP_S19_L002_R1_001,LNG4-E_CLIP_S70_L006_R2_001,618_INPUT_S48_L005_R2_001,503_INPUT_S22_L003_R1_001,KB34_293_CLIP_TIA1_Br_2_TCCGGAGA-TATAGCCT_R2,KB34_293_CLIP_TIA1_Br_2_TCCGGAGA-TATAGCCT_R1,LNG3-M_CLIP_S66_L006_R2_001,611_INPUT_S30_L003_R1_001,551_INPUT_S28_L003_R2_001,548_CLIP_S25_L003_R2_001,491_INPUT_S2_L001_R2_001,LF_CVBCB6_100_ATTACTCG_AGGCTATA_S41_L004_R2_001,LNG2-M_CLIP_S78_L007_R1_001,LNG4-M_CLIP_S72_L006_R1_001,LNG6-E_CLIP_S82_L007_R1_001,LNG4-E_CLIP_S70_L006_R1_001,KB32_293_CLIP_TIA1_Br_input_CGCTCATT-TATAGCCT_R1,KB32_293_CLIP_TIA1_Br_input_CGCTCATT-TATAGCCT_R2,LF_CVBAX6_10_S40_L004_R2_001,LNG3-E_INPUT_S61_L006_R1_001,582_INPUT_S10_L001_R2_001,613_CLIP_S43_L004_R1_001,523_CLIP_S5_L001_R2_001,548_INPUT_S26_L003_R1_001,LF_CVBCB3_100_S42_L004_R2_001,LNG2-E_INPUT_S73_L007_R2_001,LNG12-E_INPUT_S57_L006_R2_001,LNG2-E_INPUT_S73_L007_R1_001,LNG6-N_CLIP_S84_L007_R1_001,613_INPUT_S44_L004_R1_001,522